In [1]:
print('Hello LangChain')

Hello LangChain


In [7]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#print(OPENAI_API_KEY)

In [8]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("human", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="LangServe는 무엇인가요? 자세하게 설명해주세요")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [14]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001FEB0FA70E0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001FEB103A660> root_client=<openai.OpenAI object at 0x000001FEB0EC4140> root_async_client=<openai.AsyncOpenAI object at 0x000001FEB100F320> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [11]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print(response)
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

오류 발생: Error code: 400 - {'error': {'message': 'The model `mistral-saba-24b` requires terms acceptance. Please have the org admin accept the terms at https://console.groq.com/playground?model=mistral-saba-24b', 'type': 'invalid_request_error', 'code': 'model_terms_required'}}


### LCEL
* Prompt + LLM을 Chain으로 연결하기

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are an expert in AI Expert. Answer the question. 
    <Question>: {input}에 대해 쉽게 반드시 한글로 설명해주세요.")
    """)                                     
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    You are an expert in AI Expert. Answer the question. \n    <Question>: {input}에 대해 쉽게 반드시 한글로 설명해주세요.")\n    ')

In [18]:
# chain 연결 (LCEL)
chain = prompt | llm
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [ ]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL)
output_parser = StrOutputParser()

chain2 = prompt | llm | output_parser
print(type(chain2))

In [19]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리는 컴퓨터가 데이터를 통해 스스로 학습하고, 그 학습을 통해 특정 작업을 수행하는 능력을 키우는 과정입니다. 이를 쉽게 설명해 드리겠습니다.

**인공지능 모델 학습의 기본 단계**

1. **데이터 수집**: 인공지능 모델을 학습시키기 위해서는 많은 양의 데이터가 필요합니다. 이 데이터는 문제에 따라 달라지는데, 예를 들어 이미지 인식 모델을 만든다면 수많은 이미지들이 필요합니다.

2. **데이터 전처리**: 수집된 데이터를 모델이 학습할 수 있도록 가공하는 과정입니다. 이 과정에서는 데이터의 오류를 수정하거나, 데이터를 정규화하는 등의 작업이 이루어집니다.

3. **모델 선택**: 어떤 유형의 인공지능 모델을 사용할지 결정하는 단계입니다. 문제의 성격에 따라 신경망 모델, 결정 트리 모델 등 다양한 모델이 선택될 수 있습니다.

4. **학습**: 모델이 데이터를 분석하고, 스스로 규칙을 찾아내는 과정입니다. 이 과정에서는 모델이 예측을 하고, 실제 값과 예측 값의 차이를 계산합니다. 그런 다음, 모델의 매개변수들을 조정하여 예측의 정확도를 높입니다.

5. **평가**: 학습이 완료된 모델이 얼마나 잘 작동하는지 평가하는 단계입니다. 평가 데이터 세트를 사용하여 모델의 성능을 측정하고, 필요에 따라 모델을 수정합니다.

**핵심 원리: 최적화**

인공지능 모델 학습의 핵심은 최적화입니다. 최적화는 모델의 예측과 실제 값 사이의 차이를 최소화하는 과정입니다. 이를 위해 모델은 손실 함수(loss function)라는 것을 사용합니다. 손실 함수는 모델의 성능을 수치로 나타내며, 모델의 학습 목표는 이 손실 함수를 최소화하는 것입니다.

**학습의 예: 고양이 인식 모델**

예를 들어, 고양이 인식 모델을 학습시킨다고 가정해 봅시다. 이 모델은 수많은 고양이와 고양이가 아닌 것들의 이미지 데이터를 통해 학습합니다. 모델은 각 이미지를 분석하여 

In [ ]:
# chain 호출
try:
    result = chain2.invoke({"input": ": LangChain의 Products(제품)는 어떤 것들이 있나요?"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")